In [3]:
import os
import json
import pickle
import numpy as np
from tqdm import tqdm
from collections import defaultdict

In [2]:
repo_root = '/data'

## Link entities to KG

First, install the scispacy model:
```
pip install scispacy==0.3.0
pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz
```

In [ ]:
!pip install scispacy==0.3.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 23.1 MB/s eta 0:00:00
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thin

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz (33.1 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.3.0-py3-none-any.whl size=33119277 sha256=c2218b4ffddc40a4d3a6630dd35b3abaff194be9fbb13ecc849edd849d9da1fa
  Stored in directory: /root/.cache/pip/wheels/16/ad/7c/6dbe140ee4e4268cba6a5cb32da6f80e91d49c07e03d2be8ec
Successfully built en-core-sci-sm


In [ ]:
!pip install en_core_sci_sm

In [ ]:
#Load scispacy entity linker
import spacy
import scispacy
from scispacy.linking import EntityLinker
import en_core_sci_sm
def load_entity_linker(threshold=0.90):
    nlp = en_core_sci_sm.load()
    linker = EntityLinker(
        resolve_abbreviations=True,
        name="umls",
        threshold=threshold)
    nlp.add_pipe(linker)
    return nlp, linker

nlp, linker = load_entity_linker()

https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmpjk5c5j91
Finished download, copying /tmp/tmpjk5c5j91 to cache at /root/.scispacy/datasets/e9f7327283e43f0482f7c0c71b71dec278a58ccb3ffdd03c2c2350159e7ef146.f2a350ad19015b2591545f7feeed6a6d6d2fffcd635d868a5d7fc0dfc3cadfd8.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/nmslib_index.bin not found in cache, downloading to /tmp/tmpv1usl298
Finished download, copying /tmp/tmpv1usl298 to cache at /root/.scispacy/datasets/f48455d6c79262057cce66b4619123c2b558b21092d42fac97f47bb99a5b8f9f.dd70d3dffe7d90d7ac8914460e16a48375dab32485fb6313a34e6fbcaf53218b.nmslib_index.bin
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/tfidf_vectorizer.joblib not found in cache, downloading to /tmp/tmpn4juf_0s
Finished download, copying /tmp/tmpn4juf_0s to cache at /root/.scispacy/da

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/concept_aliases.json not found in cache, downloading to /tmp/tmpmn4ni39v
Finished download, copying /tmp/tmpmn4ni39v to cache at /root/.scispacy/datasets/1428ec15d3b1061731ea273c03699130b3d6b90948993e74bda66af605ff8e2a.aeb7a686c654df6bccb6c2c23d3eda3eb381daaefda4592b58158d0bee53b352.concept_aliases.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/kbs/2020-10-09/umls_2020_aa_cat0129.jsonl not found in cache, downloading to /tmp/tmpguadd6q5
Finished download, copying /tmp/tmpguadd6q5 to cache at /root/.scispacy/datasets/4d7fb8fcae1035d1e0a47d9072b43d5a628057d35497fbfb2499b4b7b2dd4dd7.05ec7eef12f336d4666da85b7fa69b9401883a7dd4244473f7b88b413ccbba03.umls_2020_aa_cat0129.jsonl
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_semantic_type_tree.tsv not found in cache, downloading to /tmp/tmp4lwd5rsv
Finished download, copying /tmp/tmp4lwd5rsv to cache at /root/.scispacy/datasets/21a1012c53

In [ ]:
def entity_linking_to_umls(sentence, nlp, linker):
    doc = nlp(sentence)
    entities = doc.ents
    all_entities_results = []
    for mm in range(len(entities)):
        entity_text = entities[mm].text
        entity_start = entities[mm].start
        entity_end = entities[mm].end
        all_linked_entities = entities[mm]._.kb_ents
        all_entity_results = []
        for ii in range(len(all_linked_entities)):
            curr_concept_id = all_linked_entities[ii][0]
            curr_score = all_linked_entities[ii][1]
            curr_scispacy_entity = linker.kb.cui_to_entity[all_linked_entities[ii][0]]
            curr_canonical_name = curr_scispacy_entity.canonical_name
            curr_TUIs = curr_scispacy_entity.types
            curr_entity_result = {"Canonical Name": curr_canonical_name, "Concept ID": curr_concept_id,
                                  "TUIs": curr_TUIs, "Score": curr_score}
            all_entity_results.append(curr_entity_result)
        curr_entities_result = {"text": entity_text, "start": entity_start, "end": entity_end, 
                                "start_char": entities[mm].start_char, "end_char": entities[mm].end_char,
                                "linking_results": all_entity_results}
        all_entities_results.append(curr_entities_result)
    return all_entities_results

In [ ]:
#Example
sent = "A 5-year-old girl is brought to the emergency department by her mother because of multiple episodes of nausea and vomiting that last about 2 hours. During this period, she has had 6–8 episodes of bilious vomiting and abdominal pain. The vomiting was preceded by fatigue."
ent_link_results = entity_linking_to_umls(sent, nlp, linker)
ent_link_results

/usr/local/lib/python3.8/dist-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/usr/local/lib/python3.8/dist-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


[{'text': 'girl',
  'start': 2,
  'end': 3,
  'start_char': 13,
  'end_char': 17,
  'linking_results': [{'Canonical Name': 'Female child',
    'Concept ID': 'C0870604',
    'TUIs': ['T100'],
    'Score': 1.0},
   {'Canonical Name': 'Woman',
    'Concept ID': 'C0043210',
    'TUIs': ['T098'],
    'Score': 1.0}]},
 {'text': 'emergency department',
  'start': 7,
  'end': 9,
  'start_char': 36,
  'end_char': 56,
  'linking_results': [{'Canonical Name': 'Accident and Emergency department',
    'Concept ID': 'C0562508',
    'TUIs': ['T073', 'T093'],
    'Score': 1.0}]},
 {'text': 'mother',
  'start': 11,
  'end': 12,
  'start_char': 64,
  'end_char': 70,
  'linking_results': [{'Canonical Name': 'Mother (person)',
    'Concept ID': 'C0026591',
    'TUIs': ['T099'],
    'Score': 1.0},
   {'Canonical Name': 'Relationship - Mother',
    'Concept ID': 'C1546508',
    'TUIs': ['T078'],
    'Score': 1.0}]},
 {'text': 'multiple',
  'start': 14,
  'end': 15,
  'start_char': 82,
  'end_char': 90,
  'l

In [ ]:
#Run entity linking to UMLS for all questions
def process(input):
    nlp, linker = load_entity_linker()
    stmts = input
    for stmt in tqdm(stmts):
        stem = stmt['question']['stem']
        stem = stem[:3500]
        stmt['question']['stem_ents'] = entity_linking_to_umls(stem, nlp, linker)
        for ii, choice in enumerate(stmt['question']['choices']):
            text = stmt['question']['choices'][ii]['text']
            stmt['question']['choices'][ii]['text_ents'] = entity_linking_to_umls(text, nlp, linker)
    return stmts

for fname in ["dev", "test", "train"]:
    with open(f"{medqa_root}/statement/{fname}.statement.jsonl") as fin:
        stmts = [json.loads(line) for line in fin]
        res = process(stmts)  
    with open(f"{medqa_root}/statement/{fname}.statement.umls_linked.jsonl", 'w') as fout:
        for dic in res:
            print (json.dumps(dic), file=fout)

In [ ]:
print('ff')

ff


In [ ]:
# Convert UMLS entity linking to DDB entity linking (our KG)
umls_to_ddb = {}
with open(f'{repo_root}/data/ddb/ddb_to_umls_cui.txt') as f:
    for line in f.readlines()[1:]:
        elms = line.split("\t")
        umls_to_ddb[elms[2]] = elms[1]

def map_to_ddb(ent_obj):
    res = []
    for ent_cand in ent_obj['linking_results']:
        CUI  = ent_cand['Concept ID']
        name = ent_cand['Canonical Name']
        if CUI in umls_to_ddb:
            ddb_cid = umls_to_ddb[CUI]
            res.append((ddb_cid, name))
    return res

def process(fname):
    with open(f"{medqa_root}/statement/{fname}.statement.umls_linked.jsonl") as fin:
        stmts = [json.loads(line) for line in fin]
    with open(f"{medqa_root}/grounded/{fname}.grounded.jsonl", 'w') as fout:
        for stmt in tqdm(stmts):
            sent = stmt['question']['stem']
            qc = []
            qc_names = []
            for ent_obj in stmt['question']['stem_ents']:
                res = map_to_ddb(ent_obj)
                for elm in res:
                    ddb_cid, name = elm
                    qc.append(ddb_cid)
                    qc_names.append(name)
            for cid, choice in enumerate(stmt['question']['choices']):
                ans = choice['text']
                ac = []
                ac_names = []
                for ent_obj in choice['text_ents']:
                    res = map_to_ddb(ent_obj)
                    for elm in res:
                        ddb_cid, name = elm
                        ac.append(ddb_cid)
                        ac_names.append(name)
                out = {'sent': sent, 'ans': ans, 'qc': qc, 'qc_names': qc_names, 'ac': ac, 'ac_names': ac_names}
                print (json.dumps(out), file=fout)     


## Load knowledge graph (KG)

Load our KG, which is based on Disease Database + DrugBank.

In [ ]:
def load_ddb():            
    with open(f'{repo_root}/data/ddb/ddb_names.json') as f:
        all_names = json.load(f)
        # print(all_names)
    with open(f'{repo_root}/data/ddb/ddb_relas.json') as f:
        all_relas = json.load(f)
    relas_lst = []
    for key, val in all_relas.items():
        relas_lst.append(val)
        
    ddb_ptr_to_preferred_name = {}
    ddb_ptr_to_name = defaultdict(list)
    ddb_name_to_ptr = {}
    for key, val in all_names.items():
        item_name = key
        item_ptr = val[0]
        item_preferred = val[1]
        if item_preferred == "1":
            ddb_ptr_to_preferred_name[item_ptr] = item_name
        ddb_name_to_ptr[item_name] = item_ptr
        ddb_ptr_to_name[item_ptr].append(item_name)
        
    return (relas_lst, ddb_ptr_to_name, ddb_name_to_ptr, ddb_ptr_to_preferred_name)


relas_lst, ddb_ptr_to_name, ddb_name_to_ptr, ddb_ptr_to_preferred_name = load_ddb()


ddb_ptr_lst, ddb_names_lst = [], []
for key, val in ddb_ptr_to_preferred_name.items():
    ddb_ptr_lst.append(key)
    ddb_names_lst.append(val)

with open(f"{repo_root}/data/ddb/vocab.txt", "w") as fout:
    for ddb_name in ddb_names_lst:
        print (ddb_name, file=fout)

with open(f"{repo_root}/data/ddb/ptrs.txt", "w") as fout:
    for ddb_ptr in ddb_ptr_lst:
        print (ddb_ptr, file=fout)

id2concept = ddb_ptr_lst
print(id2concept)

['2858', '1832', '2859', '29576', '34794', '32638', '31211', '28966', '34413', '34396', '4', '36397', '34226', '36443', '33510', '32207', '29831', '29832', '29833', '36326', '2861', '13386', '33038', '34844', '32551', '32625', '32552', '36399', '31819', '35081', '29836', '2866', '11', '34579', '29694', '32376', '31398', '2867', '30190', '29850', '2868', '2869', '2875', '16', '29329', '29332', '29945', '36626', '33155', '34101', '34545', '33683', '30046', '31181', '30819', '14326', '14367', '32854', '36668', '17', '34613', '51759', '33818', '33170', '29151', '29762', '30785', '27084', '29446', '33382', '32010', '32994', '36563', '33795', '14638', '37678', '30908', '55', '34964', '33819', '12384', '31172', '31440', '58', '34922', '62', '30626', '30598', '31195', '29946', '30563', '31789', '30564', '31790', '34572', '3056', '32866', '34205', '64', '31019', '30191', '33755', '33988', '30964', '32952', '34329', '30959', '31551', '69', '32885', '33860', '32088', '33820', '32738', '31726', '3

In [ ]:
len(ddb_ptr_to_name), len(ddb_ptr_to_preferred_name), len(ddb_name_to_ptr)

(9958, 9958, 17358)

In [ ]:
ddb_name_to_ptr['Ethanol'], ddb_name_to_ptr['Serine']

('325', '31770')

In [ ]:
merged_relations = [
    'belongs_to_the_category_of',
    'is_a_category',
    'may_cause',
    'is_a_subtype_of',
    'is_a_risk_factor_of',
    'is_associated_with',
    'may_contraindicate',
    'interacts_with',
    'belongs_to_the_drug_family_of',
    'belongs_to_drug_super-family',
    'is_a_vector_for',
    'may_be_allelic_with',
    'see_also',
    'is_an_ingradient_of',
    'may_treat'
]

relas_dict = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "6": 5, "10": 6, "12": 7, "16": 8, "17": 9, "18": 10,
             "20": 11, "26": 12, "30": 13, "233": 14}

In [ ]:
!pip install networkx --upgrade



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import networkx as nx

def construct_graph():
    concept2id = {w: i for i, w in enumerate(id2concept)}
    id2relation = merged_relations
    relation2id = {r: i for i, r in enumerate(id2relation)}
    graph = nx.MultiDiGraph()
    attrs = set()
    for relation in relas_lst:
        subj = concept2id[relation[0]]
        obj = concept2id[relation[1]]
        rel = relas_dict[relation[2]]
        weight = 1.
        graph.add_edge(subj, obj, rel=rel, weight=weight)
        attrs.add((subj, obj, rel))
        graph.add_edge(obj, subj, rel=rel + len(relation2id), weight=weight)
        attrs.add((obj, subj, rel + len(relation2id)))
    output_path = f"{repo_root}/data/ddb/ddb.graph"
    nx.write_gpickle(graph, output_path)
    return concept2id, id2relation, relation2id, graph

concept2id, id2relation, relation2id, KG = construct_graph()